In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [4]:
df_rando = pd.read_csv("Données.csv")

In [7]:
display(df_rando.info(),
        df_rando.describe(),
        df_rando.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           119 non-null    object
 1   Town           119 non-null    object
 2   Types          119 non-null    object
 3   Distance       119 non-null    object
 4   Time           119 non-null    object
 5   Difficulty     119 non-null    object
 6   Image          119 non-null    object
 7   Titre          119 non-null    object
 8   Lien_du_titre  119 non-null    object
 9   Prix           119 non-null    object
 10  Departement    119 non-null    int64 
dtypes: int64(1), object(10)
memory usage: 10.4+ KB


None

,Departement
count,119.0
mean,33.0
std,0.0
min,33.0
25%,33.0
50%,33.0
75%,33.0
max,33.0


,Name,Town,Types,Distance,Time,Difficulty,Image,Titre,Lien_du_titre,Prix,Departement
75,La Blaignanaise,blaignan-prignac,à pied,"14,2 km",Durée : 3h45/4h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,La Gazelle Perchée,https://www.cabanes-de-france.com/partenaire/l...,À partir de 200€,33
47,Boucle pédestre : A la croisée des appellations,saint-christophe-des-bardes,à pied,"11,1 km",Durée : 2h30,Difficulté : Moyenne,https://www.gironde-tourisme.com/wp-content/up...,La Roulotte des Clartés,https://www.cabanes-de-france.com/partenaire/r...,À partir de 84€,33
69,"Boucle pédestre : Tayac, Francs et Saint-Cibard",tayac,à pied,"13,4 km",Durée : 3h15,Difficulté : Difficile,https://www.gironde-tourisme.com/wp-content/up...,Le Village des Monédières,https://www.cabanes-de-france.com/partenaire/r...,À partir de 43 €€,33
81,Saint-Laurent-du-Plan boucle sud,saint-laurent-du-plan,à pied,"6,9 km",Durée : 1h10,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,Au Foudre du Bazois,https://www.cabanes-de-france.com/partenaire/a...,À partir de 70€,33
55,Boucle pédestre : Les Artigues de Lussac en t...,les artigues-de-lussac,à pied,"5,7 km",Durée : 1h45,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,Le Village de la Champagne,https://www.cabanes-de-france.com/partenaire/r...,À partir de 43€,33


# Fill NA

In [8]:
# Fill NA
df_rando[df_rando["Types"].isna()]

,Name,Town,Types,Distance,Time,Difficulty,Image,Titre,Lien_du_titre,Prix,Departement


In [9]:
# Données récupéré depuis google
df_rando["Time"][df_rando["Types"].isna()] = "Durée : 1h00"
df_rando["Difficulty"][df_rando["Types"].isna()] = "Difficulté : Facile"
df_rando["Types"][df_rando["Types"].isna()] = "à vélo"

C:\Users\moham\AppData\Local\Temp\ipykernel_11776\645246321.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_rando["Time"][df_rando["Types"].isna()] = "Durée : 1h00"
C:\Users\moham\AppData\Local\Temp\ipykernel_11776\645246321.py:2: Set

# Numerize Distance

In [10]:
df_rando["Distance"] =  df_rando["Distance"].apply(lambda x: float(x.replace(" km", "").replace(",", ".")))

# Transform Time in Duration in minutes

In [11]:
df_rando["Time"].unique()

array([' Durée : 1h30  ', ' Durée : 2h  ', ' Durée : 1h15  ',
       ' Durée : 2  ', ' Durée : 1h45  ', ' Durée : 3h  ', ' Durée : 3  ',
       ' Durée : 6h15  ', ' Durée : 3H30  ', ' Durée : 1h  ',
       ' Durée : 7h  ', ' Durée : 4  ', ' Durée : 4h00  ',
       ' Durée : 4h  ', ' Durée : 2 Jours  ', ' Durée : 1h10  ',
       ' Durée : 2h45  ', ' Durée : 5h  ', ' Durée : 30 mn  ',
       ' Durée : 2h20  ', ' Durée : 2h30  ', ' Durée : 3h30  ',
       ' Durée : 45 min  ', ' Durée : 3h00  ', ' Durée : 30mn  ',
       ' Durée : 2h10  ', ' Durée : 3h45  ', ' Durée : 1 jour  ',
       ' Durée : 7h30  ', ' Durée : 1h00  ', ' Durée : 2h15  ',
       ' Durée : 3h15  ', ' Durée : 1h20  ', ' Durée : 3,30  ',
       ' Durée : 1h50  ', ' Durée : 3h45/4h  ', ' Durée : 1H30  ',
       ' Durée : 4h30  ', ' Durée : 3h10  ', ' Durée : 45mn  ',
       ' Durée : 45 mn  ', ' Durée : 6h30  ', ' Durée : 6h10  ',
       ' Durée : 5h00  ', ' Durée : 7h00  ', ' Durée : 6h  ',
       ' Durée : 1h40  '], dtype

In [12]:
# Virer duree et mettre au même format
df_rando["Time"] = df_rando["Time"].apply(lambda x: x.replace("Durée : ", "").replace("mn", "min").replace("H", "h").replace("  ", "").replace(" ", "").replace("00", "").replace(",", "h").replace("3h45/4h", "4h").replace("1jour", "24h").replace("2Jours", "48h"))

df_rando["Time"].unique()

array(['1h30', '2h', '1h15', '2', '1h45', '3h', '3', '6h15', '3h30', '1h',
       '7h', '4', '4h', '48h', '1h10', '2h45', '5h', '30min', '2h20',
       '2h30', '45min', '2h10', '3h45', '24h', '7h30', '2h15', '3h15',
       '1h20', '1h50', '4h30', '3h10', '6h30', '6h10', '6h', '1h40'],
      dtype=object)

In [13]:
df_rando[(df_rando["Time"] == "3") | (df_rando["Time"] == "4") | (df_rando["Time"] == "2")]

,Name,Town,Types,Distance,Time,Difficulty,Image,Titre,Lien_du_titre,Prix,Departement
4,De Sainte-Foy-la-Grande à Port-Sainte-Foy-et-...,sainte-foy-la-grande,à pied,8.0,2,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,les lodges de babylone,https://www.cabanes-de-france.com/partenaire/l...,À partir de 270€,33
7,Boucle Haut Val d’Engranne Sud,gornac,à pied,10.2,3,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,Les Cabanes des Volcans,https://www.cabanes-de-france.com/partenaire/l...,À partir de 165€,33
17,La Sauve-Majeure grande boucle,la sauve,à pied,15.0,4,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,LES CABANES DE VIRE,https://www.cabanes-de-france.com/partenaire/l...,À partir de 70€,33


In [14]:
# Correction des temps incohérents
df_rando["Time"].iloc[4] = "2h"
df_rando["Time"].iloc[8] = "3h"
df_rando["Time"].iloc[21] = "4h"
df_rando["Time"].iloc[113] = "4h"

C:\Users\moham\AppData\Local\Temp\ipykernel_11776\3758786976.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_rando["Time"].iloc[4] = "2h"
C:\Users\moham\AppData\Local\Temp\ipykernel_11776\3758786976.py:2: SettingWithCopyWarning: 
A va

In [15]:
df_rando["Time"].unique()

array(['1h30', '2h', '1h15', '1h45', '3h', '3', '6h15', '3h30', '1h',
       '7h', '4', '4h', '48h', '1h10', '2h45', '5h', '30min', '2h20',
       '2h30', '45min', '2h10', '3h45', '24h', '7h30', '2h15', '3h15',
       '1h20', '1h50', '4h30', '3h10', '6h30', '6h10', '1h40'],
      dtype=object)

In [16]:
df_rando["Time"].value_counts()

Time
2h       14
3h       13
2h30     11
1h        9
1h30      8
3h30      8
4h        8
1h45      5
45min     5
5h        5
1h15      4
1h10      3
30min     3
7h        2
1h50      2
3h45      2
48h       1
2h15      1
6h10      1
6h30      1
3h10      1
4h30      1
1h20      1
3h15      1
7h30      1
2h45      1
24h       1
2h10      1
3         1
6h15      1
2h20      1
4         1
1h40      1
Name: count, dtype: int64

In [17]:
"1h10" < "1h30"

True

In [18]:
def dur(time):
    
    Duration = [{"1h": "<= 1h"}, {"2h": "<= 2h"}, {"3h": "<= 3h"}, {"5h": "<= 5h"}, {"8h": "<= 8h"}, {"48h": "> 8h"}]
    
    if time == "30min" or time == "45min":
        return "<= 1h"
    else:
        for ele in range(len(Duration)):
            if time <= [key for key in Duration[ele]][0]:
                return [val for val in Duration[ele].values()][0]

In [19]:
df_rando["Duration"] = df_rando["Time"].apply(lambda x: dur(x))

In [20]:
df_rando["Duration"].value_counts()

Duration
<= 2h    39
<= 3h    29
<= 5h    28
<= 1h    17
<= 8h     6
Name: count, dtype: int64

In [21]:
df_rando[(df_rando["Time"] == "24h") | (df_rando["Time"] == "48h")]

,Name,Town,Types,Distance,Time,Difficulty,Image,Titre,Lien_du_titre,Prix,Departement,Duration
20,Chemin de Saint-Jacques-Compostelle par la Ch...,le verdon-sur-mer,à pied,96.0,48h,Difficulté : Moyenne,https://www.gironde-tourisme.com/wp-content/up...,Le Village Toue des Demoiselles,https://www.cabanes-de-france.com/partenaire/t...,À partir de 54€,33,<= 5h
53,Urban Wine Trail,bordeaux,à pied,25.0,24h,Difficulté : Facile,https://www.gironde-tourisme.com/wp-content/up...,LE COCON DU VIVARAIS,https://www.cabanes-de-france.com/partenaire/l...,À partir de 90€,33,<= 2h


In [23]:
df_rando["Difficulty"] = df_rando["Difficulty"].apply(lambda x: x.replace(" Difficulté : ", "").replace("Difficulté : ", ""))

In [24]:
df_rando["Difficulty"].value_counts()

Difficulty
Facile          64
Moyenne         31
Très facile     20
Difficile        4
Name: count, dtype: int64

In [25]:
df_rando["Difficulty"] = df_rando["Difficulty"].apply(lambda x: x.replace("Moyenne ", "Moyenne").replace("Très Facile ", "Très Facile").replace("Facile ", "Facile").replace("Difficile ", "Difficile"))

In [26]:
display(df_rando.info(),
        df_rando.describe(),
        df_rando.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           119 non-null    object 
 1   Town           119 non-null    object 
 2   Types          119 non-null    object 
 3   Distance       119 non-null    float64
 4   Time           119 non-null    object 
 5   Difficulty     119 non-null    object 
 6   Image          119 non-null    object 
 7   Titre          119 non-null    object 
 8   Lien_du_titre  119 non-null    object 
 9   Prix           119 non-null    object 
 10  Departement    119 non-null    int64  
 11  Duration       119 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.3+ KB


None

,Distance,Departement
count,119.000000,119.0
mean,10.611429,33.0
std,10.028805,0.0
min,0.600000,33.0
25%,5.650000,33.0
50%,9.000000,33.0
75%,13.100000,33.0
max,96.000000,33.0


,Name,Town,Types,Distance,Time,Difficulty,Image,Titre,Lien_du_titre,Prix,Departement,Duration
102,Boucle de Sainte Emilionne,rimons,à pied,6.0,2h,Facile,https://www.gironde-tourisme.com/wp-content/up...,"Monchâteau Etoilé, hébergement insolite",https://www.cabanes-de-france.com/partenaire/m...,À partir de 160€,33,<= 2h
103,Sentier Street Art à Braud-et-Saint-Louis,braud-et-saint-louis,à pied,2.0,1h,Facile,https://www.gironde-tourisme.com/wp-content/up...,Cabane Spa « La Truffière » – Château de Cantecor,https://www.cabanes-de-france.com/partenaire/c...,À partir de 189€,33,<= 1h
63,La Soulacaise Sud,soulac-sur-mer,à pied,6.3,2h,Facile,https://www.gironde-tourisme.com/wp-content/up...,Les Cabanes de Fontaine,https://www.cabanes-de-france.com/partenaire/l...,À partir de 95€,33,<= 2h
50,Boucle locale de Saint-Martial,saint-martial,à pied,10.0,2h30,Moyenne,https://www.gironde-tourisme.com/wp-content/up...,BOIS DE LUNA,https://www.cabanes-de-france.com/partenaire/b...,À partir de 265€,33,<= 3h
76,Boucle du château de Villandraut,villandraut,à pied,13.8,2h30,Facile,https://www.gironde-tourisme.com/wp-content/up...,Parc de la Belle,https://www.cabanes-de-france.com/partenaire/p...,À partir de 95€,33,<= 3h


In [28]:
def rangeD(distance):
    if distance <= 6:
        return "<= 6km"
    elif distance <= 10:
        return "<= 10km"
    elif distance <= 15:
        return "<= 15km"
    else:
        return "> 15km"
    
df_rando["Range"] = df_rando["Distance"].apply(lambda x: rangeD(x))


In [29]:
X = df_rando[["Types", "Range", "Difficulty", "Duration"]]

Rando_folle = make_pipeline(OneHotEncoder(),
                            NearestNeighbors(n_neighbors = 3))
 
Rando_folle = Rando_folle.fit(X)

In [42]:
UserPref = pd.DataFrame({"Types": ["à pied"],
                         "Range": ["<= 15km"],
                         "Difficulty": ["Moyenne"],
                         "Duration": ["<= 8h"]})
UserPref

,Types,Range,Difficulty,Duration
0,à pied,<= 15km,Moyenne,<= 8h


In [44]:
closestR = Rando_folle.NearestNeighbors(UserPref, X.columns)


TypeError: 'Pipeline' object is not callable